In [ ]:
!pip install rouge datasets==1.4.1 transformers==4.4.2

## preprocess data

In [ ]:
import os

path = './demo_data/SUMMARY.meta'
os.makedirs(path, exist_ok=True)
print("The new directory is created!")

In [ ]:
## data process 
import pandas as pd
from datasets import load_dataset

df_article_summary_full = pd.read_excel('customer.xlsx', engine='openpyxl')

df_article_summary_full[['正文','摘要']].to_csv(os.path.join(path,'total.csv'),index=False)

total_data = pd.read_csv(os.path.join(path,'total.csv'))
x = total_data[-total_data['摘要'].isnull()]
x.columns = ['article','summarization']

In [ ]:
x.shape

In [ ]:
x[:int(x.shape[0]*0.7)].shape[0]+x[int(x.shape[0]*0.7):int(x.shape[0]*0.8)].shape[0]+x[int(x.shape[0]*0.8):].shape[0]

In [ ]:
# use csv file to test 
x[:int(x.shape[0]*0.7)].to_csv(os.path.join(path,'train.csv'),index=False,encoding='utf-8')
x[int(x.shape[0]*0.8):].to_csv(os.path.join(path,'test.csv'),index=False,encoding='utf-8')
x[int(x.shape[0]*0.7):int(x.shape[0]*0.8)].to_csv(os.path.join(path,'dev.csv'),index=False,encoding='utf-8')

## run train

In [ ]:
%%time
!python run_gen_v2.py --model_path 'fnlp/cpt-large' --dataset meta --data_dir demo_data --epoch '1' --batch_size '2' 

## 推理

In [ ]:
import sys
sys.path.append('../')

from transformers import BertTokenizer
from modeling_cpt import CPTModel, CPTForConditionalGeneration

model = CPTForConditionalGeneration.from_pretrained("output/meta/1")
tokenizer = BertTokenizer.from_pretrained('output/meta/1')
dataset = load_dataset('csv', data_files=os.path.join(path,'test.csv'),split='train')

In [ ]:
%%time
def print_result(idx):
    input_text = dataset[idx]['article']
    print("input: ",input_text)

    inputs = tokenizer(input_text, return_tensors="pt",max_length=512)

    #prompt_length = len(tokenizer.decode(inputs['input_ids'][0]))
    #outputs = model.generate(inputs['input_ids'], max_length=64, do_sample=True, top_p=0.95, top_k=60)
    outputs = model.generate(inputs['input_ids'], max_length=300, top_p=0.95)
    generated = tokenizer.decode(outputs[0])

    print("prediction result: ",generated)
    
    print ('label: ',dataset[idx]['summarization'])

In [ ]:
print_result(3)

## 增强训练

In [ ]:
%%time
!python run_gen_v2.py --model_path 'output/meta/1' --dataset meta --data_dir demo_data --epoch '1' --batch_size '2' 